## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-09-22-51-00 +0000,wsj,Opinion | Potheads Head for the Emergency Room,https://www.wsj.com/opinion/marijuana-referend...
1,2025-12-09-22-50-00 +0000,wsj,Opinion | France’s Great Welfare Leap Backward,https://www.wsj.com/opinion/france-pension-ref...
2,2025-12-09-22-49-50 +0000,nypost,Michigan man dies of rabies he contracted from...,https://nypost.com/2025/12/09/us-news/michigan...
3,2025-12-09-22-43-00 +0000,wsj,"One Student Killed, Another Injured in Kentuck...",https://www.wsj.com/us-news/kentucky-state-uni...
4,2025-12-09-22-37-39 +0000,nyt,"After Trump Threatens Tariffs, Mexico Seeks a ...",https://www.nytimes.com/2025/12/09/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2406/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
29,trump,41
159,new,16
339,up,14
45,out,12
468,will,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
301,2025-12-09-01-07-00 +0000,wsj,President Trump backed away from a vow to rele...,https://www.wsj.com/politics/national-security...,97
307,2025-12-09-00-29-40 +0000,nypost,Trump gives green light to Nvidia to ship powe...,https://nypost.com/2025/12/08/business/trump-g...,92
216,2025-12-09-11-56-07 +0000,nypost,"Trump warns Ukraine is ‘losing’ Russia war, ca...",https://nypost.com/2025/12/09/us-news/trump-sa...,92
291,2025-12-09-02-13-58 +0000,bbc,Trump gives Nvidia green light to sell advance...,https://www.bbc.com/news/articles/ckg9q635q6po...,86
194,2025-12-09-13-30-50 +0000,bbc,Ukraine prepares new peace plan as Zelensky ru...,https://www.bbc.com/news/articles/cg7vdd115vjo...,84


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
301,97,2025-12-09-01-07-00 +0000,wsj,President Trump backed away from a vow to rele...,https://www.wsj.com/politics/national-security...
194,84,2025-12-09-13-30-50 +0000,bbc,Ukraine prepares new peace plan as Zelensky ru...,https://www.bbc.com/news/articles/cg7vdd115vjo...
99,53,2025-12-09-19-12-00 +0000,wsj,"Bruce Blakeman, a Long Island Republican who t...",https://www.wsj.com/politics/elections/long-is...
307,51,2025-12-09-00-29-40 +0000,nypost,Trump gives green light to Nvidia to ship powe...,https://nypost.com/2025/12/08/business/trump-g...
9,48,2025-12-09-22-33-02 +0000,nypost,"Netflix, Paramount shares dive as Wall Street ...",https://nypost.com/2025/12/09/media/netflix-pa...
91,38,2025-12-09-19-40-52 +0000,nypost,Wild new cockpit audio reveals moment Alaska A...,https://nypost.com/2025/12/09/us-news/wild-new...
77,37,2025-12-09-20-34-16 +0000,nypost,Luigi Mangione utters revealing quip when he s...,https://nypost.com/2025/12/09/us-news/luigi-ma...
3,36,2025-12-09-22-43-00 +0000,wsj,"One Student Killed, Another Injured in Kentuck...",https://www.wsj.com/us-news/kentucky-state-uni...
87,36,2025-12-09-20-00-00 +0000,wsj,Kevin Hassett says he would rely on his own ju...,https://www.wsj.com/economy/central-banking/ke...
189,32,2025-12-09-14-00-00 +0000,nypost,Minnesota Lt. Gov. Peggy Flanagan failed to ho...,https://nypost.com/2025/12/09/us-news/minnesot...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
